<a href="https://colab.research.google.com/github/catalinamagalvan/PySimMIBCI/blob/main/Load_data_ftp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!git clone https://github.com/catalinamagalvan/PySimMIBCI.git

fatal: destination path 'PySimMIBCI' already exists and is not an empty directory.


In [15]:
!pip install mne
!pip install fooof
!pip install colorednoise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
from scipy.io import loadmat
import os
import mne
import numpy as np
from mne.datasets import fetch_fsaverage
import sys
sys.path.insert(1, os.path.join('PySimMIBCI', 'utils'))
from datasets import load_and_epoch_OpenBMI_by_ftp, create_OpenBMI_info, raw_from_OpenBMI_data
from FOOOF import fit_user_params, plot_FOOOF
from simulation import generate_when, generate_what, generate_where
from noise import add_aperiodic_activity, add_eye_movement
from plots import plot_raw_2_channels

Set parameters of data simulation

In [17]:
# MI IDs
MI_tasks_IDs = {'MI/left': 1, 'MI/right': 2}
# Sample frequency
sfreq = 1000
# Number of trials
N_trials = 50
# MI trial duration in ms
MI_duration = 4500
# ERD percentage
reduction = 0.5

Download real MI-EEG data from session 1 of subject 27 of OpenBMI dataset and create corresponding MNE Epochs object.

In [18]:
epochs_real, epochs_real_right, epochs_real_left = load_and_epoch_OpenBMI_by_ftp(subject=27,
                                                                                 session=1,
                                                                                 epoch_window=[0, 4],
                                                                                 picks=None)

Generate suitable mne Info object for OpenBMI data.

In [19]:
info = create_OpenBMI_info()

Fit aperiodic and periodic parameters of real data.

In [20]:
user_params = fit_user_params(epochs_real, epochs_real_right, epochs_real_left)

/content/PySimMIBCI/utils/FOOOF.py:84: UserWarning: Zero value in spectrum for channel TP10
  PSD_left, freqs = epochs_left.compute_psd(method='multitaper',
/content/PySimMIBCI/utils/FOOOF.py:88: UserWarning: Zero value in spectrum for channel TP10
  PSD_both, freqs_both = epochs.compute_psd(method='multitaper',


Setup source space and compute forward solution.

In [24]:
# Download head model (fsaverage) files
fs_dir = fetch_fsaverage(verbose=False)
subjects_dir = os.path.dirname(fs_dir)
src = os.path.join(fs_dir, 'bem', 'fsaverage-ico-5-src.fif')
bem = os.path.join(fs_dir, 'bem', 'fsaverage-5120-5120-5120-bem-sol.fif')
subject = 'fsaverage'
trans = 'fsaverage'  # MNE has a built-in fsaverage transformation
fwd = mne.make_forward_solution(info, trans=trans, src=src,
                                bem=bem, eeg=True, mindist=5.0)
# Here, :class:`~mne.simulation.SourceSimulator` is used, which allows to
# specify where (label), what (source_time_series), and when (events) an
# event type will occur.
src = fwd['src']
source_simulator = mne.simulation.SourceSimulator(src, tstep=1/sfreq)

Generate when, what and where information for the task-related component in right hand MI vs left hand MI scenario.

In [25]:
events_info = {0: {'label': 'MI/left', 'duration': MI_duration},
               1: {'label': 'MI/right', 'duration': MI_duration}}
when = generate_when(events_info, N_trials, sfreq)

what = generate_what(MI_tasks_IDs.keys(), when, user_params, MI_duration,
                     sfreq, N_trials, reduction)
where = generate_where(subject, subjects_dir)

Add task-related component to source simulator object.

In [26]:
for label in what.keys():
    for task_ID, task in enumerate(MI_tasks_IDs.keys(), 1):
        when_task = when[np.where(when[:, 2] == task_ID)[0]]
        source_simulator.add_data(label=where[label],
                                  waveform=what[label][task],
                                  events=when_task)

Simulate raw data. Project the source time series to sensor space.
The `mne.simulation.SourceSimulator` object can be given directly to the `mne.simulation.simulate_raw` function.

In [ ]:
raw_sim = mne.simulation.simulate_raw(info, source_simulator, forward=fwd)
raw_sim.set_eeg_reference(ref_channels='average')

Add aperiodic activity (background noise) to raw simulated data. Here, user-specific aperiodic parameters are employed.

In [ ]:
raw_sim = add_aperiodic_activity(raw_sim,
                                 exponent=user_params['aperiodic_params'][1],
                                 offset=user_params['aperiodic_params'][0])

Add eog artifacts by MNE function

In [ ]:
mne.simulation.add_eog(raw_sim)

Add eye-movement artifacts

In [ ]:
add_eye_movement(raw_sim)

Epoch data

In [ ]:
epochs_sim = mne.Epochs(raw_sim, when, MI_tasks_IDs, tmin=0, tmax=4,
                        baseline=None)
epochs_sim_right = epochs_sim['MI/right']
epochs_sim_left = epochs_sim['MI/left']

FOOOF analysis

In [ ]:
f_min = 1
f_max = 40
t_min = 0
t_max = 3
ch_names = epochs_sim.ch_names
PSD_sim_right, freqs = epochs_sim_right.compute_psd(method='multitaper',
                                                    fmin=f_min, fmax=f_max,
                                                    tmin=t_min,
                                                    tmax=t_max).get_data(
                                                        return_freqs=True)
PSD_sim_left, freqs = epochs_sim_left.compute_psd(method='multitaper',
                                                  fmin=f_min, fmax=f_max,
                                                  tmin=t_min,
                                                  tmax=t_max).get_data(
                                                        return_freqs=True)

plot_FOOOF(freqs, np.mean(PSD_sim_right[:, ch_names.index('C3')], axis=0))
plot_FOOOF(freqs, np.mean(PSD_sim_right[:, ch_names.index('C4')], axis=0))
plot_FOOOF(freqs, np.mean(PSD_sim_left[:, ch_names.index('C3')], axis=0))
plot_FOOOF(freqs, np.mean(PSD_sim_left[:, ch_names.index('C4')], axis=0))

PSD_real_right, freqs = epochs_real_right.compute_psd(method='multitaper',
                                                      fmin=f_min, fmax=f_max,
                                                      tmin=t_min,
                                                      tmax=t_max).get_data(
                                                        return_freqs=True)
PSD_real_left, freqs = epochs_real_left.compute_psd(method='multitaper',
                                                    fmin=f_min, fmax=f_max,
                                                    tmin=t_min,
                                                    tmax=t_max).get_data(
                                                        return_freqs=True)
ch_names = epochs_real.ch_names
plot_FOOOF(freqs, np.mean(PSD_real_right[:, ch_names.index('C3')], axis=0))
plot_FOOOF(freqs, np.mean(PSD_real_right[:, ch_names.index('C4')], axis=0))
plot_FOOOF(freqs, np.mean(PSD_real_left[:, ch_names.index('C3')], axis=0))
plot_FOOOF(freqs, np.mean(PSD_real_left[:, ch_names.index('C4')], axis=0))

Plot some EEG channels

In [ ]:
# Real data
raw_real = raw_from_OpenBMI_data(fname)
raw_real.filter(l_freq=0.05, h_freq=200, picks='eeg', method='iir',
                phase='zero')
plot_raw_2_channels(raw_real, start=20, duration=20)

# Simulated data
raw_sim.filter(l_freq=0.05, h_freq=200, picks='eeg', method='iir',
               phase='zero')
plot_raw_2_channels(raw_sim, start=20, duration=20)